# Labeling the [Civil Comments](https://huggingface.co/datasets/civil_comments) dataset using Autolabel

This dataset contains public comments collected from news websites, the task is a binary classification task -- is the provided comment toxic or not

## Install Autolabel
Plus, setup your OpenAI API key, since we'll be using gpt-3.5-turbo as our LLM for labeling.

In [1]:
!pip3 install 'refuel-autolabel[openai]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.4/770.4 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74

In [29]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxxxxxxx'

## Download the dataset

In [2]:
from autolabel import get_data

get_data('civil_comments')

This downloads two datasets:

* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!
Labeling with Autolabel is a 3-step process:

* First, we specify a labeling configuration (see `config` object below)
* Next, we do a dry-run on our dataset using the LLM specified in `config` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

### Experiment #1: Very simple guidelines

In [3]:
from autolabel import LabelingAgent

In [12]:
config = {
    "task_name": "ToxicCommentClassification",
    "task_type": "classification", # classification task
    "dataset": {
        "label_column": "label",
    },
    "model": {
        "provider": "openai",
        "name": "gpt-3.5-turbo" # the model we want to use
    },
    "prompt": {
        "task_guidelines": "Does the provided comment contain 'toxic' language? Say toxic or not toxic.",
        "labels": [ # list of labels to choose from
            "toxic",
            "not toxic"
        ],
        "example_template": "Input: {example}\nOutput: {label}"
    }
}


Let's review the configuration file above. You'll notice the following useful keys:

* `task_type`: `classification` (since it's a classification task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.task_guidelines`: Is the provided comment 'toxic' or 'not toxic'? (how we describe the task to the LLM)
* `prompt.labels`: ['toxic', 'not toxic'] (the two labels to choose from)

In [13]:
# create an agent for labeling
agent = LabelingAgent(config=config)

In [14]:
# dry-run -- this tells us how much this will cost and shows an example prompt
agent.plan('test.csv')

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $4.3332 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0022 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

Does the provided comment contain 'toxic' language? Say toxic or not toxic.

You will return the answer with just one element: "the correct label"

Now I want you to label the following example:
Input: [ Integrity means that you pay your debts.]

Does this apply to President Trump too?
Output: 


───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [15]:
# now, do the actual labeling
labels, df, metrics = agent.run('test.csv', max_items=100)

There is an existing task with following details: id='847339966' created_at=datetime.datetime(2023, 6, 23, 4, 44, 
23, 797679) task_id='085adce3c7879b879eaa67455b235c04' dataset_id='0727e8d6821f8e518bff55de3eba0718' 
current_index=100 output_file='test_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None metrics=None

Evaluating the existing task...

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 100     │ -inf      │ 0.56     │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

100 examples labeled so far.

───────────────────────────────────────────── Last Annotated Example ──────────────────────────────────────────────

...

y


Output()

Actual Cost: 0.0


┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 100     │ -inf      │ 0.56     │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0

### Experiment #2: Few-shot prompting to provide helpful examples

In [16]:
config = {
    "task_name": "ToxicCommentClassification",
    "task_type": "classification",
    "dataset": {
        "label_column": "label",
        "delimiter": ","
    },
    "model": {
        "provider": "openai",
        "name": "gpt-3.5-turbo",
    },
    "prompt": {
        "task_guidelines":  "Does the provided comment contain 'toxic' language? Say toxic or not toxic.",
        "labels": [
            "toxic",
            "not toxic"
        ],
        "few_shot_examples": [
            {
                "example": "It's ridiculous that these guys are being called 'protesters'. Being armed is a threat of violence, which makes them terrorists.",
                "label": "toxic"
            },
            {
                "example": "This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",
                "label": "not toxic"
            },
            {
                "example": "This bitch is nuts. Who would read a book by a woman",
                "label": "toxic"
            },
            {
                "example": "It was a great show. Not a combo I'd of expected to be good together but it was.",
                "label": "not toxic"
            }
        ],
        "few_shot_selection": "fixed",
        "few_shot_num": 4,
        "example_template": "Input: {example}\nOutput: {label}"
    }
}


In [17]:
# create an agent for labeling
agent = LabelingAgent(config, cache=False)

In [18]:
# dry-run -- this tells us how much this will cost and shows an example prompt
agent.plan(dataset='test.csv')

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $4.7081 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0024 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

Does the provided comment contain 'toxic' language? Say toxic or not toxic.

You will return the answer with just one element: "the correct label"

Some examples with their output answers are provided below:

Input: It's ridiculous that these guys are being called 'protesters'. Being armed is a threat of violence, which makes them terrorists.
Output: toxic

Input: This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!
Output: not toxic

Input: This bitch is nuts. Who would read a book by a woman
Output: toxic

Input: It was a great show. Not a combo I'd of expected to be good together but it was.
Output: not toxic

Now I want you to label the following example:
Input: [ Integrity means that you pay your debts.]

Does this apply to President Trump too?
Output: 


───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [19]:
_, output_df, _ = agent.run('test.csv', max_items=100)

Output()

Actual Cost: 0.0358


┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 100     │ -inf      │ 0.69     │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0

### Experiment #3: Improving prompts with error analysis (Prompt engineering)

`agent.run()` returned an output Pandas Dataframe. We'll review the mistakes and update the labeling guidelines to "teach" the LLM our content moderation policies.

In [ ]:
import pandas as pd

pd.set_option('max_colwidth', None)

In [ ]:
# review top 10 mistakes
display(output_df[output_df['label'] != output_df['ToxicCommentClassification_llm_label']].head(10))

,example,label,ToxicCommentClassification_llm_labeled_successfully,ToxicCommentClassification_llm_label
1,This is malfeasance by the Administrator and the Board. They are wasting our money!,not toxic,True,toxic
2,"@Rmiller101 - Spoken like a true elitist. But look out bud. The re-awakening in Europe, Brexit and now the Trump victory are just the beginning. The unwashed masses are coming for you and your ilk….",not toxic,True,toxic
4,Sorry you missed high school. Eisenhower sent troops to Vietnam after the French withdrew in 1954 and before that America was providing 50% of the cost of that war. WWI & WWII were won by Democrat Presidents and the last win a Republican had was 1865 but the did surrender in Korea and Vietnam and fail to win in desert Storm or these two wars.,not toxic,True,toxic
5,Let’s see if I understand this; Berkowitz announces a $14M surplus then he rails against Proposition 8 because it would constrain his ability to confiscate even more money from citizens. Berkowitz and his allies on the Assembly changed the tax cap language to facilitate confiscating greater amounts of citizen’s money. Now why would a mayor who professes fidelity with the taxpayers change the law to enable him to take more money from them? Probably because he plans to take more money from them. Berkowitz said if Proposition 8 passes “it severely constrains our ability to give out property tax relief.” So Berkowitz wants the ability to confiscate more of the citizen’s money to provide property relief. How about taking less of that money in the first place? Let’s hope Proposition 8 passes.,not toxic,True,toxic
8,What would Jerry Prevo think about that story? Exactly what I think about his....,not toxic,True,toxic
9,When wil the indigenous be accountable and take responsibility instead of blaming others ? They have to be accountable and pay their own way or go it alone.,not toxic,True,toxic
10,the people against O'Leary are the same people against\n\nTrump\nBrexit\n\nnot a good track record so far\n2 down and 1 to go,not toxic,True,toxic
12,"Since whining is the lifeblood of online comment sites, the Pope's sign will find little support here. The self-pitying ones with their myriad complaints would have you believe that Canada is a dreadful place to live -- why they haven't left for their perfect foreign paradise is a mystery -- when by any relative standard it remains a refuge of stability in a chaotic world, its ongoing issues not likely to be solved by endless griping from anonymous online cranks.",not toxic,True,toxic
15,IT IS DOA! As Collins announced her intentions to vote NO!,not toxic,True,toxic
23,"Kitty Piercy did her best to make the almighty ""legacy"" (city) hall larger-than-life, at the city council work session on 7-11. Move the slide time-bar to 11:18 minutes, on the webcast of that meeting:\n\nhttp://ceapps.eugene-or.gov/CEWebcast/WebCast/Play.aspx?mid=2996",not toxic,True,toxic


Based on the mistakes we are making, we will make some adjustments to our `task_guidelines`.

In [20]:
config = {
    "task_name": "ToxicCommentClassification",
    "task_type": "classification",
    "dataset": {
        "label_column": "label",
        "delimiter": ","
    },
    "model": {
        "provider": "openai",
        "name": "gpt-3.5-turbo",
    },
    "prompt": {
        "task_guidelines": "You are an expert at identifying toxic comments. You aim to act in a fair and balanced manner, where comments that provide fair criticism of something or someone are labelled 'not toxic'. Similarly, criticisms of policy and politicians are marked 'not toxic', unless the comment includes obscenities, racial slurs or sexually explicit material. Any comments that are sexually explicit, obscene, or insults a person, demographic or race are not allowed and labeled 'toxic'. \nYour job is to correctly label the provided input example into one of the following categories:\n{labels}",
        "labels": [
            "toxic",
            "not toxic"
        ],
        "few_shot_examples": [
            {
                "example": "It's ridiculous that these guys are being called 'protesters'. Being armed is a threat of violence, which makes them terrorists.",
                "label": "toxic"
            },
            {
                "example": "This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",
                "label": "not toxic"
            },
            {
                "example": "This bitch is nuts. Who would read a book by a woman",
                "label": "toxic"
            },
            {
                "example": "It was a great show. Not a combo I'd of expected to be good together but it was.",
                "label": "not toxic"
            }
        ],
        "few_shot_selection": "fixed",
        "few_shot_num": 4,
        "example_template": "Input: {example}\nOutput: {label}"
    }
}


In [21]:
# create an agent for labeling
agent = LabelingAgent(config, cache=False)

In [22]:
# dry-run -- this tells us how much this will cost and shows an example prompt
agent.plan(dataset='test.csv')

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $5.0021 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0025 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at identifying toxic comments. You aim to act in a fair and balanced manner, where comments that provide fair criticism of something or someone are labelled 'not toxic'. Similarly, criticisms of policy and politicians are marked 'not toxic', unless the comment includes obscenities, racial slurs or sexually explicit material. Any comments that are sexually explicit, obscene, or insults a person, demographic or race are not allowed and labeled 'toxic'. 
Your job is to correctly label the provided input example into one of the following categories:
toxic
not toxic

You will return the answer with just one element: "the correct label"

Some examples with their output answers are provided below:

Input: It's ridiculous that these guys are being called 'protesters'. Being armed is a threat of violence, which makes them terrorists.
Output: toxic

Input: This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!
Output: not toxic

In

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [23]:
# now, do the actual labeling
_, output_df, _ = agent.run('test.csv', max_items=100)

Output()

Actual Cost: 0.0505


┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 100     │ -inf      │ 0.77     │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0

### Experiment #4: Using a different LLM

We've iterated a fair bit on prompts, and few-shot examples. Let's evaluate a few different LLMs provided by the library out of the box. For example, we observe that we can boost performance even further by using `text-davinci-003`

In [25]:
config = {
    "task_name": "ToxicCommentClassification",
    "task_type": "classification",
    "dataset": {
        "label_column": "label",
        "delimiter": ","
    },
    "model": {
        "provider": "openai",
        "name": "text-davinci-003",
    },
    "prompt": {
        "task_guidelines": "You are an expert at identifying toxic comments. You aim to act in a fair and balanced manner, where comments that provide fair criticism of something or someone are labelled 'not toxic'. Similarly, criticisms of policy and politicians are marked 'not toxic', unless the comment includes obscenities, racial slurs or sexually explicit material. Any comments that are sexually explicit, obscene, or insults a person, demographic or race are not allowed and labeled 'toxic'. \nYour job is to correctly label the provided input example into one of the following categories:\n{labels}",
        "labels": [
            "toxic",
            "not toxic"
        ],
        "few_shot_examples": [
            {
                "example": "It's ridiculous that these guys are being called 'protesters'. Being armed is a threat of violence, which makes them terrorists.",
                "label": "toxic"
            },
            {
                "example": "This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",
                "label": "not toxic"
            },
            {
                "example": "This bitch is nuts. Who would read a book by a woman",
                "label": "toxic"
            },
            {
                "example": "It was a great show. Not a combo I'd of expected to be good together but it was.",
                "label": "not toxic"
            }
        ],
        "few_shot_selection": "fixed",
        "few_shot_num": 4,
        "example_template": "Input: {example}\nOutput: {label}"
    }
}


In [26]:
# create an agent for labeling
agent = LabelingAgent(config, cache=False)

In [27]:
# dry-run -- this tells us how much this will cost and shows an example prompt
agent.plan(dataset='test.csv')

Output()

┌──────────────────────────┬──────────┐
│ Total Estimated Cost     │ $53.9556 │
│ Number of Examples       │ 2000     │
│ Average cost per example │ $0.027   │
└──────────────────────────┴──────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at identifying toxic comments. You aim to act in a fair and balanced manner, where comments that provide fair criticism of something or someone are labelled 'not toxic'. Similarly, criticisms of policy and politicians are marked 'not toxic', unless the comment includes obscenities, racial slurs or sexually explicit material. Any comments that are sexually explicit, obscene, or insults a person, demographic or race are not allowed and labeled 'toxic'. 
Your job is to correctly label the provided input example into one of the following categories:
toxic
not toxic

You will return the answer with just one element: "the correct label"

Some examples with their output answers are provided below:

Input: It's ridiculous that these guys are being called 'protesters'. Being armed is a threat of violence, which makes them terrorists.
Output: toxic

Input: This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!
Output: not toxic

In

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [28]:
# now, do the actual labeling
_, output_df, _ = agent.run('test.csv', max_items=100)

Output()

Actual Cost: 0.7022


┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 100     │ -inf      │ 0.88     │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0

### Experiment #5: Using confidence scores

In [ ]:
# Start computing confidence scores (using Refuel's LLMs)
os.environ['REFUEL_API_KEY'] = 'sk-xxxxxxxxxxxx'

In [ ]:
config = {
    "task_name": "ToxicCommentClassification",
    "task_type": "classification",
    "dataset": {
        "label_column": "label",
        "delimiter": ","
    },
    "model": {
        "provider": "openai",
        "name": "gpt-3.5-turbo",
        "compute_confidence": True,
    },
    "prompt": {
        "task_guidelines": "You are an expert at identifying toxic comments. You aim to act in a fair and balanced manner, where comments that provide fair criticism of something or someone are labelled 'not toxic'. Similarly, criticisms of policy and politicians are marked 'not toxic', unless the comment includes obscenities, racial slurs or sexually explicit material. Any comments that are sexually explicit, obscene, or insults a person, demographic or race are not allowed and labeled 'toxic'. \nYour job is to correctly label the provided input example into one of the following categories:\n{labels}",
        "labels": [
            "toxic",
            "not toxic"
        ],
        "few_shot_examples": [
            {
                "example": "It's ridiculous that these guys are being called 'protesters'. Being armed is a threat of violence, which makes them terrorists.",
                "label": "toxic"
            },
            {
                "example": "This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",
                "label": "not toxic"
            },
            {
                "example": "This bitch is nuts. Who would read a book by a woman",
                "label": "toxic"
            },
            {
                "example": "It was a great show. Not a combo I'd of expected to be good together but it was.",
                "label": "not toxic"
            }
        ],
        "few_shot_selection": "fixed",
        "few_shot_num": 4,
        "example_template": "Input: {example}\nOutput: {label}"
    }
}


In [ ]:
# create an agent for labeling
agent = LabelingAgent(config, cache=False)

In [ ]:
# dry-run -- this tells us how much this will cost and shows an example prompt
agent.plan('test.csv')

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $4.7442 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0024 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────


You are an expert at identifying toxic comments.

You aim to act in a fair and balanced manner, where comments that provide fair criticism of something or someone are labelled 'not toxic'. Similarly, criticisms of policy and politicians are marked 'not toxic', unless the comment includes obscenities, racial slurs or sexually explicit material.

Any comments that are explicit, obscene, or insults a person, demographic or race are not allowed and labeled 'toxic'.


You will return the answer with just one element: "the correct label"

Now I want you to label the following example:
Input: [ Integrity means that you pay your debts.]

Does this apply to President Trump too?
Output: 


───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
# now, do the actual labeling
_, output_df, _ = agent.run('test.csv', start_index=0, max_items=100)

There is an existing task with following details: id='4132226781' created_at=datetime.datetime(2023, 6, 15, 17, 47,
3, 646196) task_id='b45559d2c3b770648f92f7134a76ee30' dataset_id='0727e8d6821f8e518bff55de3eba0718' 
current_index=10 output_file='test_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None metrics=None

Evaluating the existing task...

Metric: auroc: 0.5


┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 12      │ -inf      │ 0.6667   │ 1.0             │
│ 12      │ 0.5       │ 0.6667   │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

12 examples have been labeled so far.

...

Deleted the existing task and starting a new one...

Output()

Metric: auroc: 0.8875
Actual Cost: 0.0376


┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 100     │ -inf      │ 0.75     │ 1.0             │
│ 1       │ 0.9988    │ 1.0      │ 0.01            │
│ 12      │ 0.9957    │ 1.0      │ 0.12            │
│ 13      │ 0.9949    │ 0.9231   │ 0.13            │
│ 54      │ 0.9128    │ 0.9815   │ 0.54            │
│ 55      │ 0.9107    │ 0.9636   │ 0.55            │
│ 63      │ 0.6682    │ 0.9683   │ 0.63            │
│ 69      │ 0.6669    │ 0.8841   │ 0.69            │
│ 70      │ 0.6667    │ 0.8857   │ 0.7             │
│ 72      │ 0.6667    │ 0.8611   │ 0.72            │
│ 76      │ 0.4935    │ 0.8684   │ 0.76            │
│ 77      │ 0.4908    │ 0.8571   │ 0.77            │
│ 79      │ 0.4819    │ 0.8608   │ 0.79            │
│ 80      │ 0.4774    │ 0.85     │ 0.8             │
│ 87      │ 0.4423    │ 0.8621   │ 0.87            │
│ 100     │ 0.0402    │ 0.75     │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0